# 해당 파일(50lv_maker.ipynb)의 목적

지난 절차에서는 주어진 데이터셋에서 포켓몬 배틀에 필요하리라 생각되는 정보만을 추출했습니다. 이 파일을 통해서는 주어진 정보를 가공해 보다 적합한 수치를 추출하고자 합니다.

## 포켓몬의 능력치 산정

포켓몬 배틀의 승패에는 다양한 요인이 영향을 미칩니다. 제한된 시간 내에 분석을 진행하기 위해 몇 가지 조건을 가정하였습니다.

우선적으로 포켓몬 배틀의 승패에 영향일 미치는 요인과 제가 한 가정을 기술하고자 합니다.

### 1. 기술

포켓몬에는 다양한 기술이 존재합니다. 각각의 기술을 위력과 타입이 다릅니다. 애니메이션에서 피카츄가 쓰는 전광석화는 위력이 40인 반면에, 거북왕이 쓰는 하이드로펌프는 위력이 120입니다.

그러나 이러한 다양한 기술과 기술의 조합을 파악해 분석을 진행하기에는 시간이 부족하여 기술의 모두 50이라고 가정하여 분석을 진행했습니다.

#### 1.1 기술의 구분

기술은 크게 타입과 공격 유형(물리, 특수)로 나뉩니다. 타입은 물, 비행, 전기 등 포켓몬의 타입과 동일합니다. 예를 들어 피카츄는 전기 포켓몬이지만 노말 타입인 전광석화의 기술을 사용합니다. 이렇듯 타입의 종류는 포켓몬과 기술이 공유합니다.

결론적으로 기술은 전기, 물 등의 18가지 타입* (물리, 특수) = 36가지로 나뉩니다.

#### 1.2 기술의 효과 무시

일반적으로 물리 공격은 공격력 수치에, 특수 공격은 특수공경력 수치에 영향을 받습니다.

또한 기술은 다양한 효과를 갖고 있습니다. 예를 들어 전기자석파 같은 기술은 상대에게 데미지를 주지 않지만 마비를 일으키며, 잠자기 기술은 잠에 들며 체력을 회복시키는 기술입니다. 또한 전광적화는 포켓몬의 스피드에 상관없이 무조건 상대보다 먼저 공격이 들어갑니다.

이렇듯 기술은 데미지 외에도 다양한 효과를 발휘하지만, 간단하게 분석을 진행하기 위해 기술의 모든 효과를 무시했습니다.

### 2. 특성, 성격, 노력치, 개체값, 날씨, 아이템 효과 고려하지 않음

###### 특성 : http://ko.pokemon.wikia.com/wiki/%ED%8A%B9%EC%84%B1
###### 성격 : http://ko.pokemon.wikia.com/wiki/%EC%84%B1%EA%B2%A9
###### 노력치 : http://ko.pokemon.wikia.com/wiki/%EB%85%B8%EB%A0%A5%EC%B9%98
###### 개체값 : http://ko.pokemon.wikia.com/wiki/%EA%B0%9C%EC%B2%B4%EA%B0%92
###### 날씨 : http://ko.pokemon.wikia.com/wiki/%EB%82%A0%EC%94%A8
##### 참고 : 포켓몬 위키

포켓몬에서는 기술의 효과 외에도 다양한 요인이 포켓몬 배틀에 영향을 미칩니다. 분석을 진행하기 위해 특성, 성격, 노력치, 개체값, 날씨, 아이템 효과를 모두 무시하였습니다.

포켓몬의 특성과 성격은 고려하지 않았고, 노력치와 개체값은 모두 0을 가정했습니다.

### 3. 분석을 진행하기 위한 포켓몬 조건 동일화

모든 포켓몬의 레벨을 50으로 가정하였습니다.

### 4. 포켓몬 능력치 산출

###### http://ko.pokemon.wikia.com/wiki/%EB%8A%A5%EB%A0%A5%EC%B9%98
##### 참고 : 포켓몬 위키

In [1]:
# 분석에 필요한 패키치를 호출합니다.

import numpy as np
import pandas as pd

In [2]:
# pre_edit.ipynb에서 만든 파일을 불러옵니다.

pokemon_stat = pd.read_csv("pokemon_stat.csv", encoding = 'utf-8')

In [3]:
# 데이터 확인

pokemon_stat.head()

pokedex_number        name  generation  is_legendary   classfication  \
0               1   Bulbasaur           1             0    Seed Pokémon   
1               2     Ivysaur           1             0    Seed Pokémon   
2               3    Venusaur           1             0    Seed Pokémon   
3               4  Charmander           1             0  Lizard Pokémon   
4               5  Charmeleon           1             0   Flame Pokémon   

   type1   type2  hp  speed  attack      ...        against_ghost  \
0  grass  poison  45     45      49      ...                  1.0   
1  grass  poison  60     60      62      ...                  1.0   
2  grass  poison  80     80     100      ...                  1.0   
3   fire     NaN  39     65      52      ...                  1.0   
4   fire     NaN  58     80      64      ...                  1.0   

   against_grass  against_ground  against_ice  against_normal  against_poison  \
0           0.25             1.0          2.0             1.0             1.0   
1           0.25             1.0          2.0             1.0             1.0   
2           0.25             1.0          2.0             1.0             1.0   
3           0.50             2.0          0.5             1.0             1.0   
4           0.50             2.0          0.5             1.0             1.0   

   against_psychic  against_rock  against_steel  against_water  
0              2.0           1.0            1.0            0.5  
1              2.0           1.0            1.0            0.5  
2              2.0           1.0            1.0            0.5  
3              1.0           2.0            0.5            2.0  
4              1.0           2.0            0.5            2.0  

[5 rows x 31 columns]

In [4]:
# 포켓몬 공식에 맞춰 50 레벨을 기준으로 한 능력치를 산출합니다.

pokemon_stat['50lv_hp'] = pokemon_stat.apply(lambda x: x['hp'] + 50 if x['generation'] < 3 else x['hp'] + 60, axis = 1)
pokemon_stat['50lv_speed'] = pokemon_stat.apply(lambda x: x['speed'] + 55 if x['generation'] < 3 else x['speed'] + 5, axis = 1)
pokemon_stat['50lv_attack'] = pokemon_stat.apply(lambda x: x['attack'] + 55 if x['generation'] < 3 else x['attack'] + 5, axis = 1)
pokemon_stat['50lv_defense'] = pokemon_stat.apply(lambda x: x['defense'] + 55 if x['generation'] < 3 else x['defense'] + 5, axis = 1)
pokemon_stat['50lv_sp_attack'] = pokemon_stat.apply(lambda x: x['sp_attack'] + 55 if x['generation'] < 3 else x['sp_attack'] + 5, axis = 1)
pokemon_stat['50lv_sp_defense'] = pokemon_stat.apply(lambda x: x['sp_defense'] + 55 if x['generation'] < 3 else x['sp_defense'] + 5, axis = 1)

In [5]:
# 데이터 확인

pokemon_stat.head()

pokedex_number        name  generation  is_legendary   classfication  \
0               1   Bulbasaur           1             0    Seed Pokémon   
1               2     Ivysaur           1             0    Seed Pokémon   
2               3    Venusaur           1             0    Seed Pokémon   
3               4  Charmander           1             0  Lizard Pokémon   
4               5  Charmeleon           1             0   Flame Pokémon   

   type1   type2  hp  speed  attack       ...         against_psychic  \
0  grass  poison  45     45      49       ...                     2.0   
1  grass  poison  60     60      62       ...                     2.0   
2  grass  poison  80     80     100       ...                     2.0   
3   fire     NaN  39     65      52       ...                     1.0   
4   fire     NaN  58     80      64       ...                     1.0   

   against_rock  against_steel  against_water  50lv_hp  50lv_speed  \
0           1.0            1.0            0.5       95         100   
1           1.0            1.0            0.5      110         115   
2           1.0            1.0            0.5      130         135   
3           2.0            0.5            2.0       89         120   
4           2.0            0.5            2.0      108         135   

   50lv_attack  50lv_defense  50lv_sp_attack  50lv_sp_defense  
0          104           104             120              120  
1          117           118             135              135  
2          155           178             177              175  
3          107            98             115              105  
4          119           113             135              120  

[5 rows x 37 columns]

In [6]:
# columns를 정렬하기 위해 재정렬합니다.

columns = list(pokemon_stat.columns)
col = columns[0:7] + columns[-6:] + columns[13:-6]

In [7]:
# 재정렬된 column 확인

col

['pokedex_number',
 'name',
 'generation',
 'is_legendary',
 'classfication',
 'type1',
 'type2',
 '50lv_hp',
 '50lv_speed',
 '50lv_attack',
 '50lv_defense',
 '50lv_sp_attack',
 '50lv_sp_defense',
 'against_bug',
 'against_dark',
 'against_dragon',
 'against_electric',
 'against_fairy',
 'against_fight',
 'against_fire',
 'against_flying',
 'against_ghost',
 'against_grass',
 'against_ground',
 'against_ice',
 'against_normal',
 'against_poison',
 'against_psychic',
 'against_rock',
 'against_steel',
 'against_water']

In [8]:
# 데이터 저장

pokemon_stat_50lv = pokemon_stat[col]
pokemon_stat_50lv.to_csv("pokemon_stat_50lv.csv", encoding = "utf-8", index = False)